In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import os
import time
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler
import json
import warnings
warnings.filterwarnings('ignore')


def normalize(s, sensitive_att, data):
    size = list(data[sensitive_att].value_counts())
    for idx, line in enumerate(s):
        s[idx] = line / sum(line) * size[idx]
    return np.cumsum(np.around(s, 0), axis=1)

def get_distribution_index(alpha, dataset, nb_user, sensitive_att, nb_class):
    indiv_list = []
    for goal in range(0, nb_class):
        list_1 = [idx for idx, x in enumerate(dataset[sensitive_att]) if x == goal]
        indiv_list.append(list_1)
    np.random.seed(9)
    s = normalize(np.random.dirichlet([alpha] * nb_class, nb_user).transpose(), sensitive_att, dataset)
    data_list_transfer = []
    for user in range(0, nb_user):
        if user == 0:
            bound_1 = 5
            bound_2 = int(s[0][user])
            tmp = indiv_list[0][bound_1:bound_2]
            data_list_transfer.append(tmp)
        else:
            bound_1 = int(s[0][user - 1])
            bound_2 = int(s[0][user])
            tmp = indiv_list[0][bound_1:bound_2]
            data_list_transfer.append(tmp)
        for class_ in range(1, 2):
            if user == 0:
                bound_1 = 0
                bound_2 = int(s[class_][user])
                tmp = indiv_list[class_][bound_1:bound_2]
                data_list_transfer[user] = data_list_transfer[user] + tmp
            else:
                bound_1 = int(s[class_][user - 1])
                bound_2 = int(s[class_][user])
                tmp = indiv_list[class_][bound_1:bound_2]
                data_list_transfer[user] = data_list_transfer[user] + tmp
    dict_users = {}
    for i in range(nb_user):
        dict_users[i] = set(data_list_transfer[i])
    return dict_users


def generate_client_data_dirichelet(alpha, df, nb_clients, sensitive_att, nb_class):
    dfs = []
    dist = get_distribution_index(alpha, df, nb_clients, sensitive_att, nb_class)
    for i in range(len(dist)):
        dfs.append(df.iloc[list(dist[i]), :])
    j = 0
    for i in range(len(dfs)):
        if dfs[i].size == 0:
           #print("the number of generated datasets is less than the number of clients")
           j = j+1
    return dfs

In [ ]:
# read data
# train = pd.read_csv("path/to/trainset/KDD.csv")

alpha = 0.02
nb_users = 5
sensitive_att_name = "age"
nb_classes = 2


train_sets = generate_client_data_dirichelet(alpha, train, nb_users, sensitive_att_name, nb_classes)